In [2]:
import requests
import json
from pprint import pprint
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from config import apikey
import pandas as pd

--------------------------------------------code below used for testing---------------------------------------------------------

In [2]:
api_key= apikey
headers = {'Authorization': 'Bearer %s' % api_key}

In [ ]:
url='https://api.yelp.com/v3/businesses/search'
 
# In the dictionary, term can take values like food, cafes or businesses like McDonalds
params = {'term':'bars','location':'San Francisco'}

In [ ]:
# Making a get request to the API
req=requests.get(url, params=params, headers=headers)
 
# proceed only if the status code is 200
print('The status code is {}'.format(req.status_code))

In [ ]:
# printing the text from the response 
json.loads(req.text)

In [ ]:
req_json = req.json()
pprint(req_json)

In [ ]:
out_file = open("sfbars.json", "w")  
    
json.dump(req_json, out_file, indent = 6)  
    
out_file.close() 

In [ ]:
url='https://api.yelp.com/v3/businesses/search'
 
# In the dictionary, term can take values like food, cafes or businesses like McDonalds
params = {'term':'bars','location':'San Francisco', 'offset':51}

In [ ]:
req2=requests.get(url, params=params, headers=headers)

In [ ]:
req2_json = req2.json()
pprint(req2_json)

In [ ]:
out_file = open("sfbars2.json", "w")  
    
json.dump(req2_json, out_file, indent = 6)  
    
out_file.close() 

In [ ]:
import glob
import json

file_list = glob.glob("*.json")
file_list

In [ ]:
allFilesDict = {v:k for v, k in enumerate(file_list, 1)}
allFilesDict

In [ ]:
data = []

for k,v in allFilesDict.items():
    if 1 <= k <= 2:
        with open(v, 'r') as d:
            jdata = json.load(d)
            if jdata:
                data.append(jdata)

In [ ]:
type(data)

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.to_json(r'data_output_default.json',orient='records')

--------------------------------------------code above used for testing---------------------------------------------------------

---------------------------------------final code used for outputs below--------------------------------------------------------

In [3]:
def search_bars(set_num):
    url = 'https://api.yelp.com/v3/businesses/search'
    api_key=apikey
    headers = {'Authorization': 'Bearer %s' % api_key}
    url_params = { #parameters passed to the API
    "categories": 'bars, ALL',
    "location":"San Francisco",
    "state": "California",
    'offset': offset_num, # We are going to iterate the offset
     "limit":50 # Maximum return of results per request (ref: API documentation).
     }

    response = requests.get(url, headers=headers, params=url_params)
    return response.json()

In [4]:
# !pip install --user pandas==1.0.3
# import pandas as pd
# pd.__version__

In [6]:
for offset_num in np.arange(0,1000,50) : 
# I want up to 550 results, in steps of 50 results per request.
#     try:
    output_json = search_bars(offset_num) # Executing the function defined above.
#     print(offset_num) # Making sure each offset iteration is running
#     pprint(output_json) # If you wanna check the JSON for each iteration
    if offset_num == 0:
       
        df_first = pd.json_normalize(output_json['businesses'], errors='ignore')
#         print("Worked in 0")
# 'businesses' because that's the primary key of the JSON (i.e. pull all attribute data by calling 
# that one key). This is something you can figure out reading the API documentation or visually
# parsing the JSON. 
    else:
#             df2 = pd.json_normalize(output_json['businesses'])
        df2 = pd.json_normalize(output_json['businesses'], errors='ignore')
        df_first = df_first.append(df2)
#         print("Worked else")

#     except AttributeError:
#         print("error at ", offset_num) # Helpful for debugging purposes

In [7]:
df_first

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,coordinates.latitude,coordinates.longitude,location.address1,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address
0,oT08T3Vpn1I7jDmrBBRMTw,house-of-prime-rib-san-francisco,House of Prime Rib,https://s3-media4.fl.yelpcdn.com/bphoto/HLrjaM...,False,https://www.yelp.com/biz/house-of-prime-rib-sa...,7209,"[{'alias': 'tradamerican', 'title': 'American ...",4.0,[],...,37.793380,-122.422500,1906 Van Ness Ave,,,San Francisco,94109,US,CA,"[1906 Van Ness Ave, San Francisco, CA 94109]"
1,f-m7-hyFzkf0HSEeQ2s-9A,fog-harbor-fish-house-san-francisco-2,Fog Harbor Fish House,https://s3-media1.fl.yelpcdn.com/bphoto/5wuA-0...,False,https://www.yelp.com/biz/fog-harbor-fish-house...,6094,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.0,[restaurant_reservation],...,37.809041,-122.410233,The Embarcadero,Ste A-202,,San Francisco,94133,US,CA,"[The Embarcadero, Ste A-202, San Francisco, CA..."
2,ttarnopezxmp2ROB1N2PaA,nopa-san-francisco,Nopa,https://s3-media3.fl.yelpcdn.com/bphoto/16cGmQ...,False,https://www.yelp.com/biz/nopa-san-francisco?ad...,5076,"[{'alias': 'newamerican', 'title': 'American (...",4.0,[delivery],...,37.774830,-122.437460,560 Divisadero St,None,,San Francisco,94117,US,CA,"[560 Divisadero St, San Francisco, CA 94117]"
3,KkAVX3Wb7E3lP5F_f_8pXg,liholiho-yacht-club-san-francisco-2,Liholiho Yacht Club,https://s3-media2.fl.yelpcdn.com/bphoto/dHWRdc...,False,https://www.yelp.com/biz/liholiho-yacht-club-s...,2421,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,[delivery],...,37.788180,-122.414580,871 Sutter St,,,San Francisco,94109,US,CA,"[871 Sutter St, San Francisco, CA 94109]"
4,mSMZJj2pFvttWLpcDmgrEA,tonys-pizza-napoletana-san-francisco,Tony's Pizza Napoletana,https://s3-media1.fl.yelpcdn.com/bphoto/6jgpZM...,False,https://www.yelp.com/biz/tonys-pizza-napoletan...,5094,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,[delivery],...,37.800332,-122.409054,1570 Stockton St,,,San Francisco,94133,US,CA,"[1570 Stockton St, San Francisco, CA 94133]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,obbj78gbuT-mSwe5i5G8ng,san-francisco-new-years-eve-san-francisco,San Francisco New Years Eve,https://s3-media2.fl.yelpcdn.com/bphoto/aIAF5V...,False,https://www.yelp.com/biz/san-francisco-new-yea...,1,"[{'alias': 'pubs', 'title': 'Pubs'}]",5.0,[],...,37.797760,-122.431910,1980 Union St,,,San Francisco,94123,US,CA,"[1980 Union St, San Francisco, CA 94123]"
46,yQ7xPB632J5a9NYtDKDl7g,jp-morgan-club-san-francisco,JP Morgan Club,,False,https://www.yelp.com/biz/jp-morgan-club-san-fr...,1,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",5.0,[],...,37.767920,-122.387390,1 Warriors Way,,None,San Francisco,94158,US,CA,"[1 Warriors Way, San Francisco, CA 94158]"
47,GvLEvVvNAN7v1ePyoFFKnA,bar-217-san-francisco,Bar 217,https://s3-media3.fl.yelpcdn.com/bphoto/yeBwkF...,False,https://www.yelp.com/biz/bar-217-san-francisco...,1,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",5.0,[],...,37.797195,-122.406120,217 Columbus St,None,,San Francisco,94133,US,CA,"[217 Columbus St, San Francisco, CA 94133]"
48,aUeBNa2W1GXjAeXmSM4nHw,bristol-bar-and-grill-san-francisco,Bristol Bar & Grill,https://s3-media4.fl.yelpcdn.com/bphoto/yKVbij...,False,https://www.yelp.com/biz/bristol-bar-and-grill...,12,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",2.0,[],...,37.806435,-122.418068,1300 Columbus Ave,,,San Francisco,94133,US,CA,"[1300 Columbus Ave, San Francisco, CA 94133]"


In [7]:
df_first = df_first.rename(columns={"coordinates.latitude": "latitude", "coordinates.longitude": "longitude", "location.display_address": "display_address"})
df_first

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,latitude,longitude,location.address1,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,display_address
0,oT08T3Vpn1I7jDmrBBRMTw,house-of-prime-rib-san-francisco,House of Prime Rib,https://s3-media4.fl.yelpcdn.com/bphoto/HLrjaM...,False,https://www.yelp.com/biz/house-of-prime-rib-sa...,7209,"[{'alias': 'tradamerican', 'title': 'American ...",4.0,[],...,37.793380,-122.422500,1906 Van Ness Ave,,,San Francisco,94109,US,CA,"[1906 Van Ness Ave, San Francisco, CA 94109]"
1,f-m7-hyFzkf0HSEeQ2s-9A,fog-harbor-fish-house-san-francisco-2,Fog Harbor Fish House,https://s3-media1.fl.yelpcdn.com/bphoto/5wuA-0...,False,https://www.yelp.com/biz/fog-harbor-fish-house...,6094,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.0,[restaurant_reservation],...,37.809041,-122.410233,The Embarcadero,Ste A-202,,San Francisco,94133,US,CA,"[The Embarcadero, Ste A-202, San Francisco, CA..."
2,ttarnopezxmp2ROB1N2PaA,nopa-san-francisco,Nopa,https://s3-media3.fl.yelpcdn.com/bphoto/16cGmQ...,False,https://www.yelp.com/biz/nopa-san-francisco?ad...,5076,"[{'alias': 'newamerican', 'title': 'American (...",4.0,[delivery],...,37.774830,-122.437460,560 Divisadero St,None,,San Francisco,94117,US,CA,"[560 Divisadero St, San Francisco, CA 94117]"
3,KkAVX3Wb7E3lP5F_f_8pXg,liholiho-yacht-club-san-francisco-2,Liholiho Yacht Club,https://s3-media2.fl.yelpcdn.com/bphoto/dHWRdc...,False,https://www.yelp.com/biz/liholiho-yacht-club-s...,2421,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,[delivery],...,37.788180,-122.414580,871 Sutter St,,,San Francisco,94109,US,CA,"[871 Sutter St, San Francisco, CA 94109]"
4,mSMZJj2pFvttWLpcDmgrEA,tonys-pizza-napoletana-san-francisco,Tony's Pizza Napoletana,https://s3-media1.fl.yelpcdn.com/bphoto/6jgpZM...,False,https://www.yelp.com/biz/tonys-pizza-napoletan...,5094,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,[delivery],...,37.800332,-122.409054,1570 Stockton St,,,San Francisco,94133,US,CA,"[1570 Stockton St, San Francisco, CA 94133]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,obbj78gbuT-mSwe5i5G8ng,san-francisco-new-years-eve-san-francisco,San Francisco New Years Eve,https://s3-media2.fl.yelpcdn.com/bphoto/aIAF5V...,False,https://www.yelp.com/biz/san-francisco-new-yea...,1,"[{'alias': 'pubs', 'title': 'Pubs'}]",5.0,[],...,37.797760,-122.431910,1980 Union St,,,San Francisco,94123,US,CA,"[1980 Union St, San Francisco, CA 94123]"
46,yQ7xPB632J5a9NYtDKDl7g,jp-morgan-club-san-francisco,JP Morgan Club,,False,https://www.yelp.com/biz/jp-morgan-club-san-fr...,1,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",5.0,[],...,37.767920,-122.387390,1 Warriors Way,,None,San Francisco,94158,US,CA,"[1 Warriors Way, San Francisco, CA 94158]"
47,GvLEvVvNAN7v1ePyoFFKnA,bar-217-san-francisco,Bar 217,https://s3-media3.fl.yelpcdn.com/bphoto/yeBwkF...,False,https://www.yelp.com/biz/bar-217-san-francisco...,1,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",5.0,[],...,37.797195,-122.406120,217 Columbus St,None,,San Francisco,94133,US,CA,"[217 Columbus St, San Francisco, CA 94133]"
48,aUeBNa2W1GXjAeXmSM4nHw,bristol-bar-and-grill-san-francisco,Bristol Bar & Grill,https://s3-media4.fl.yelpcdn.com/bphoto/yKVbij...,False,https://www.yelp.com/biz/bristol-bar-and-grill...,12,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",2.0,[],...,37.806435,-122.418068,1300 Columbus Ave,,,San Francisco,94133,US,CA,"[1300 Columbus Ave, San Francisco, CA 94133]"


In [8]:
df_first.to_csv("bars_data_output.csv", index = False)

In [9]:
#Database setup

rds_connection_string = "postgres:postgres@localhost:5432/bars_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [11]:
#Load data

df_first = df_first.applymap(str)
df_first.to_sql(name='bars_sf', con=engine, if_exists='replace', index=True)

In [13]:
bars_metadata = "bars_data_output.csv"

In [14]:
bars_metadata = pd.read_csv(bars_metadata)
bars_metadata

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,latitude,longitude,location.address1,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,display_address
0,oT08T3Vpn1I7jDmrBBRMTw,house-of-prime-rib-san-francisco,House of Prime Rib,https://s3-media4.fl.yelpcdn.com/bphoto/HLrjaM...,False,https://www.yelp.com/biz/house-of-prime-rib-sa...,7210,"[{'alias': 'tradamerican', 'title': 'American ...",4.0,[],...,37.793380,-122.422500,1906 Van Ness Ave,NaN,NaN,San Francisco,94109.0,US,CA,"['1906 Van Ness Ave', 'San Francisco, CA 94109']"
1,f-m7-hyFzkf0HSEeQ2s-9A,fog-harbor-fish-house-san-francisco-2,Fog Harbor Fish House,https://s3-media1.fl.yelpcdn.com/bphoto/5wuA-0...,False,https://www.yelp.com/biz/fog-harbor-fish-house...,6092,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.0,['restaurant_reservation'],...,37.809041,-122.410233,The Embarcadero,Ste A-202,NaN,San Francisco,94133.0,US,CA,"['The Embarcadero', 'Ste A-202', 'San Francisc..."
2,ttarnopezxmp2ROB1N2PaA,nopa-san-francisco,Nopa,https://s3-media3.fl.yelpcdn.com/bphoto/16cGmQ...,False,https://www.yelp.com/biz/nopa-san-francisco?ad...,5076,"[{'alias': 'newamerican', 'title': 'American (...",4.0,['delivery'],...,37.774830,-122.437460,560 Divisadero St,NaN,NaN,San Francisco,94117.0,US,CA,"['560 Divisadero St', 'San Francisco, CA 94117']"
3,KkAVX3Wb7E3lP5F_f_8pXg,liholiho-yacht-club-san-francisco-2,Liholiho Yacht Club,https://s3-media2.fl.yelpcdn.com/bphoto/dHWRdc...,False,https://www.yelp.com/biz/liholiho-yacht-club-s...,2421,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,['delivery'],...,37.788180,-122.414580,871 Sutter St,NaN,NaN,San Francisco,94109.0,US,CA,"['871 Sutter St', 'San Francisco, CA 94109']"
4,mSMZJj2pFvttWLpcDmgrEA,tonys-pizza-napoletana-san-francisco,Tony's Pizza Napoletana,https://s3-media1.fl.yelpcdn.com/bphoto/6jgpZM...,False,https://www.yelp.com/biz/tonys-pizza-napoletan...,5094,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,['delivery'],...,37.800332,-122.409054,1570 Stockton St,NaN,NaN,San Francisco,94133.0,US,CA,"['1570 Stockton St', 'San Francisco, CA 94133']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,obbj78gbuT-mSwe5i5G8ng,san-francisco-new-years-eve-san-francisco,San Francisco New Years Eve,https://s3-media2.fl.yelpcdn.com/bphoto/aIAF5V...,False,https://www.yelp.com/biz/san-francisco-new-yea...,1,"[{'alias': 'pubs', 'title': 'Pubs'}]",5.0,[],...,37.797760,-122.431910,1980 Union St,NaN,NaN,San Francisco,94123.0,US,CA,"['1980 Union St', 'San Francisco, CA 94123']"
996,yQ7xPB632J5a9NYtDKDl7g,jp-morgan-club-san-francisco,JP Morgan Club,NaN,False,https://www.yelp.com/biz/jp-morgan-club-san-fr...,1,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",5.0,[],...,37.767920,-122.387390,1 Warriors Way,NaN,NaN,San Francisco,94158.0,US,CA,"['1 Warriors Way', 'San Francisco, CA 94158']"
997,GvLEvVvNAN7v1ePyoFFKnA,bar-217-san-francisco,Bar 217,https://s3-media3.fl.yelpcdn.com/bphoto/yeBwkF...,False,https://www.yelp.com/biz/bar-217-san-francisco...,1,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",5.0,[],...,37.797195,-122.406120,217 Columbus St,NaN,NaN,San Francisco,94133.0,US,CA,"['217 Columbus St', 'San Francisco, CA 94133']"
998,aUeBNa2W1GXjAeXmSM4nHw,bristol-bar-and-grill-san-francisco,Bristol Bar & Grill,https://s3-media4.fl.yelpcdn.com/bphoto/yKVbij...,False,https://www.yelp.com/biz/bristol-bar-and-grill...,12,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",2.0,[],...,37.806435,-122.418068,1300 Columbus Ave,NaN,NaN,San Francisco,94133.0,US,CA,"['1300 Columbus Ave', 'San Francisco, CA 94133']"


In [17]:
# Data Exploartion

bars_metadata.columns

Index(['id', 'alias', 'name', 'image_url', 'is_closed', 'url', 'review_count',
       'categories', 'rating', 'transactions', 'price', 'phone',
       'display_phone', 'distance', 'latitude', 'longitude',
       'location.address1', 'location.address2', 'location.address3',
       'location.city', 'location.zip_code', 'location.country',
       'location.state', 'display_address'],
      dtype='object')

In [18]:
# Data Exploration

# rating_count = bars_metadata['rating'].value_counts()
# rating_count

In [19]:
# Creating new DataFrame (1)

list(bars_metadata.columns)
new_columns = [
    'name',
    'review_count',
    'categories',
    'rating',
]

In [20]:
# Creating new DataFrame (1)

categories_data = bars_metadata[new_columns]

In [22]:
# Creating new DataFrame (2)

# translating JSON object to Python dictionary in category column
# nexted loop to make new DataFrame (category, subcategory1, subcategory 2)

categories = pd.DataFrame()
max_cat = 0
#categories = []
row = {}
# 1 category, 2 sub
for index in range(0, len(categories_data['categories'])):
    test = json.JSONDecoder().decode(categories_data['categories'][index].replace("'", '"'))
    if len(test) > max_cat:
        max_cat = len(test)
    for n in range(0, len(test)):
        if n == 0:
            row['category'] = test[n]['title']
        elif n == 1:
            row['subcategory1'] = test[n]['title']
        else:
            row['subcategory2'] = test[n]['title']
    categories = categories.append(row, ignore_index=True)
categories.head()

,category,subcategory1,subcategory2
0,American (Traditional),Steakhouses,Wine Bars
1,Seafood,Bars,Wine Bars
2,American (New),Desserts,Cocktail Bars
3,Bars,American (New),Seafood
4,Pizza,Italian,Cocktail Bars


In [23]:
#merge with original dataset

bars_metadata = bars_metadata.join(categories, how='outer')
bars_metadata.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,display_address,category,subcategory1,subcategory2
0,oT08T3Vpn1I7jDmrBBRMTw,house-of-prime-rib-san-francisco,House of Prime Rib,https://s3-media4.fl.yelpcdn.com/bphoto/HLrjaM...,False,https://www.yelp.com/biz/house-of-prime-rib-sa...,7210,"[{'alias': 'tradamerican', 'title': 'American ...",4.0,[],...,NaN,NaN,San Francisco,94109.0,US,CA,"['1906 Van Ness Ave', 'San Francisco, CA 94109']",American (Traditional),Steakhouses,Wine Bars
1,f-m7-hyFzkf0HSEeQ2s-9A,fog-harbor-fish-house-san-francisco-2,Fog Harbor Fish House,https://s3-media1.fl.yelpcdn.com/bphoto/5wuA-0...,False,https://www.yelp.com/biz/fog-harbor-fish-house...,6092,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.0,['restaurant_reservation'],...,Ste A-202,NaN,San Francisco,94133.0,US,CA,"['The Embarcadero', 'Ste A-202', 'San Francisc...",Seafood,Bars,Wine Bars
2,ttarnopezxmp2ROB1N2PaA,nopa-san-francisco,Nopa,https://s3-media3.fl.yelpcdn.com/bphoto/16cGmQ...,False,https://www.yelp.com/biz/nopa-san-francisco?ad...,5076,"[{'alias': 'newamerican', 'title': 'American (...",4.0,['delivery'],...,NaN,NaN,San Francisco,94117.0,US,CA,"['560 Divisadero St', 'San Francisco, CA 94117']",American (New),Desserts,Cocktail Bars
3,KkAVX3Wb7E3lP5F_f_8pXg,liholiho-yacht-club-san-francisco-2,Liholiho Yacht Club,https://s3-media2.fl.yelpcdn.com/bphoto/dHWRdc...,False,https://www.yelp.com/biz/liholiho-yacht-club-s...,2421,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,['delivery'],...,NaN,NaN,San Francisco,94109.0,US,CA,"['871 Sutter St', 'San Francisco, CA 94109']",Bars,American (New),Seafood
4,mSMZJj2pFvttWLpcDmgrEA,tonys-pizza-napoletana-san-francisco,Tony's Pizza Napoletana,https://s3-media1.fl.yelpcdn.com/bphoto/6jgpZM...,False,https://www.yelp.com/biz/tonys-pizza-napoletan...,5094,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,['delivery'],...,NaN,NaN,San Francisco,94133.0,US,CA,"['1570 Stockton St', 'San Francisco, CA 94133']",Pizza,Italian,Cocktail Bars


In [25]:
#Load data

bars_metadata = bars_metadata.applymap(str)
bars_metadata.to_sql(name='bars_sf', con=engine, if_exists='replace', index=True)

In [26]:
# Creating new DataFrame (3)

category_count = bars_metadata['category'].value_counts()
category_count

Bars              130
Cocktail Bars      80
Dive Bars          79
American (New)     75
Lounges            73
                 ... 
Sports Clubs        1
Wineries            1
Salad               1
Cuban               1
Hot Pot             1
Name: category, Length: 97, dtype: int64

In [27]:
# Creating new DataFrame (3)

category_count = pd.DataFrame(bars_metadata["category"].value_counts()).reset_index()
category_count

,index,category
0,Bars,130
1,Cocktail Bars,80
2,Dive Bars,79
3,American (New),75
4,Lounges,73
...,...,...
92,Sports Clubs,1
93,Wineries,1
94,Salad,1
95,Cuban,1


In [28]:
#Load data

category_count.to_sql(name='category_count', con=engine, if_exists='append', index=True)

In [ ]:
top5_category_count = pd.DataFrame(bars_metadata["category"].value_counts()[:5]).reset_index()
top5_category_count

In [ ]:
next5_category_count = pd.DataFrame(bars_metadata["category"].value_counts()[6:11]).reset_index()
next5_category_count

In [ ]:
category_count.to_csv("category_count.csv", index = False)
next5_category_count.to_csv("next5_category_count.csv", index = False)

In [ ]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
session = Session(engine)

In [ ]:
res= session.execute('SELECT * FROM "bars_sf"')

In [ ]:
for x in res:
    print(x)

In [ ]:
with open('bars_data_output3.json', 'w') as f:
    f.write(df_first.to_json(orient='records', lines=True))

In [ ]:
df_first.to_json(r'bars_data_output2.json',orient='split')